# Recipe Shopping List

This notebook generates a shopping list for recipe ingredient list specified in a spreadsheet saved as TSV (tab).

It is recommended that you modify the route listing the order of ingredients along a path.

I like to copy/paste the output list into Google Keep so I can check things off on my phone as I move through the store.

**Be careful that the strings in the TSV are not double quoted.**

See the installation instructions for the [.NET Jupyter kernel](https://github.com/dotnet/interactive/blob/master/docs/NotebooksLocalExperience.md).

In [1]:
open System.IO
open System.Text.RegularExpressions

type RecipeIngredient =
    {
        Recipe : string
        Ingredient : string
        Quantity : float
        Unit : string
        Location : float
    } with 
    static member Create(recipe, ingredient, quantity, unit, location ) = { Recipe = recipe; Ingredient = ingredient; Quantity = quantity; Unit = unit; Location = location }

//matches a numeric decimal quantity followed by a non numeric unit identifier, e.g. "1.5oz"
let quantityRegex = new Regex( @"([0-9\.]+)\s*(\D+)")

//Map of recipe name to list of ingredients
let db = 
    Directory.GetFiles(".","*.tsv").[0] //will take the first file we find
    |> File.ReadAllLines
    |> Array.skip 1                     //skip header
    |> Array.map( fun row ->
        let s = row.Split('\t')
        
        //last col mixes quantity and unit
        let m = quantityRegex.Match(s.[2])
        
        //if we have quantity and unit (e.g. 1.5oz)
        if m.Groups.Count = 3 then
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=m.Groups.[1].Value|>float; Unit=m.Groups.[2].Value; Location = s.[3] |> float }
        //unit is identity (e.g. 1 carrot)
        else
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=s.[2]|>float; Unit=""; Location = s.[3] |> float}
    )
    |> Array.groupBy( fun ri -> ri.Recipe )
    |> Map.ofArray
    
//display the recipes we know about
let format ( s : string ) = s.Replace(";",";\n").Replace("\"\"","\"")
db |> Map.toArray |> Array.map fst |> sprintf "%A" |> format

The below script needs to be able to find the current output cell; this is an easy method to get it.

[|"baked ziti chickpeas";
 "cabbage cheddar pie";
 "celery curry";
 "chana masala";

  "cheese chilli enchilladas";
 "cream of lentil soup";
 "cuban black beans";

  "edamame fried rice";
 "edamame noodles";
 "koshari";

  "lentils with fruit and sweet potatoes";
 "okra curry";
 "pasta puttanesca";

  "penne all'arrabbiata";
 "portabello mushroom ragout";
 "potato chilli burro";

  "red pepper gnocchi";
 "spinach enchiladas verde";
 "spinach lasagne";

  "spinach lentil soup";
 "sweet potato beetroot curry";

  "sweet potato black bean quessadillas";
 "sweet potato burritos";

  "szechuan tofu broccoli mushrooms";
 "toovar ke malai";

  "two step southwestern stew";
 "veggie burros";
 "zucchini tostadas"|]

**Copy the list above below and remove the recipes you don't want**
(TODO checkbox GUI)

In [2]:
let recipes =
 [|
    "baked ziti chickpeas";
    //"cabbage cheddar pie";
     //"celery curry";
    "chana masala";

      //"cheese chilli enchilladas";
    // "cream of lentil soup";
      "cuban black beans";

      "edamame fried rice";
//      "edamame noodles";
//     "koshari";

      //"lentils with fruit and sweet potatoes";
//      "okra curry";
     "pasta puttanesca";

      //"penne all'arrabbiata";
//      "portabello mushroom ragout";
//      "potato chilli burro";

//       "red pepper gnocchi";
      "spinach enchiladas verde";
//      "spinach lasagne";

//       "spinach lentil soup";
     //"sweet potato beetroot curry";

//       "sweet potato black bean quessadillas";
//      "sweet potato burritos";

      "szechuan tofu broccoli mushrooms";
     //"toovar ke malai";

      //"two step southwestern stew";
      "veggie burros";
//      "zucchini tostadas"
 |]
 
//if sequence route through store is available, use it for sorting
let routeMap = 
    if File.Exists("route") then
        File.ReadAllLines("route") |> Array.mapi( fun i x -> x,i) |> Map.ofArray
    else
        Map.empty
let routeOrder ingredient =
    match routeMap.TryFind ingredient with
    | Some(i) -> i
    | None -> 
        printfn "No route for %A" ingredient
        0 //put unknown items at top of list

// given selected recipes, collect all ingredients as groups, and sum their quantities
let ingredients =
    recipes
    
    //index by ingredient
    |> Array.collect( fun r ->
        db.[r] |> Array.map( fun ri -> ri.Ingredient,ri )
    )
    |> Array.sortBy( fun (ingredient,ri) -> 
        //minimally sort by approximate section of store
        if routeMap.IsEmpty then
            ri.Location
        //ideally sort by path through store
        else
            ingredient |> routeOrder |> float)
    |> Array.groupBy fst

    //aggregate quantities for each ingredient as much as possilbe
    |> Array.map( fun (i,tuples) -> 
    
        //sum quantities of the same unit
        let unitQuantities = 
            tuples
            |> Array.map snd
            |> Array.groupBy( fun ri2 -> ri2.Unit ) 
            |> Array.map( fun (u,ris2) ->
                (ris2 |> Array.sumBy( fun ri2 -> ri2.Quantity ) |> string) + " " + u
            )
        
        //map to string
        i + ":" + (unitQuantities |> String.concat "; ")
    )

printfn "-----------"
printfn "%A" (ingredients |> String.concat "\n")
printfn "-----------"
printfn "%A" (recipes |> String.concat "\n")

-----------
"cremini mushrooms:8 oz
garlic:12 clove
ginger:2 tb
carrot:6 
spinach:16 oz
green onions:9 
cilantro:0.5 c
broccoli:1 ; 1 c
onion:4 
zucchini:3 
green bell pepper:3 
yellow bell pepper:1 
chillis:2 
banana:3 
lemon:1 
stock cubes:2 
kalamata olives:0.25 c; 4 tb
green olives:4 tb
capers:2 tb
penne pasta:8 oz
linguine:16 oz
chickpeas:3 can
black beans:3 can
chili paste with garlic:1 tb
soy sauce:0.25 c; 4 tb
rice:3.5 c
green chillis:4 oz
tortillas:15 
green salsa:7 oz
salsa:1 c
crushed tomatoes:2 can
cooking sherry:0.25 c
tomatoes:2 can
tomato puree:1 c
tomato paste:1 tb
tomatoes basil garlic oregano:1 can
breadcrumbs:2 tb
curry powder:2 t
corn starch:2 tb
tumeric:1 t
oregano:1.5 t
salt:0.5 t
garlic salt:0.5 t
chilli powder:1 t
corriander:1 tb
cumin:2 t; 1 tb
red pepper flakes:0.5 t
garam masala:1 t
olive oil:4 tb
vegetable oil:11 tb
cheese:4 c
extra firm tofu:16 oz
sour cream:1 c
edamame:18 oz"
-----------
"baked ziti chickpeas
chana masala
cuban black beans
edamame fried ri